In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [5]:
#Link Digimap
#main_link = 'https://shopee.co.id/shop/255563049/search'
main_link = 'https://shopee.com.my/miya_gadget?categoryId=100013&entryPoint=ShopByPDP&itemId=21522121109&upstream=search'
path = r'C:\Users\syakir1937\Task Scheduler\Learn Programming\chromedriver.exe'

#Customize chrome display
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--headless')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')

driver = webdriver.Chrome(executable_path= path, options=chrome_options)
driver.get(main_link)

C:\Users\syakir1937\AppData\Local\Temp\ipykernel_24100\4042212539.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= path, options=chrome_options)


In [6]:
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html,"html.parser")

In [7]:
#Product Name
product_name = soup.findAll('div', class_='_89o+uF mC2b-a d-wZla')
product_name[0].get_text()

'Vivo Y19 (8GB + 128GB ROM) 6.53 Inch 16MP Triper Camera LTE (New) Full Set With 1 Year Warranty Original SmartPhones'

In [8]:
#Product Price
product_price = soup.findAll('div', class_='_1cMhqd tdQs-h')
product_price[0].get_text()


'RM398.00'

In [9]:
#Total sold products/month
product_sold = soup.findAll('div', class_='r4l-mr R9rrSp')
product_sold[0].get_text()

'765 sold'

In [10]:
#Looping
product_name, product_price, product_sold = [], [], []

#Product Name
for i in soup.findAll('div', class_='_89o+uF mC2b-a d-wZla'):
    product_name.append(i.text)

#Product Price
for i in soup.findAll('div', class_='_1cMhqd tdQs-h'):
    product_price.append(i.text)

#Total sold products/month
for i in soup.findAll('div', class_='r4l-mr R9rrSp'):
    product_sold.append(i.text)

In [11]:
#Many pages
product_name, product_price, product_sold = [], [], []

for page in range(0,2):
    #main_link= 'https://shopee.co.id/shop/255563049/search?page={}&sortBy=pop'.format(page)
    main_link= 'https://shopee.com.my/miya_gadget?categoryId=100013&itemId=21522121109&page={}&sortBy=pop&upstream=search'.format(page)
    driver.get(main_link)

    #Product Name
    for i in soup.findAll('div', class_='_89o+uF mC2b-a d-wZla'):
        product_name.append(i.text)

    #Product Price
    for i in soup.findAll('div', class_='_1cMhqd tdQs-h'):
        product_price.append(i.text)

    #Total sold products/month
    for i in soup.findAll('div', class_='r4l-mr R9rrSp'):
        product_sold.append(i.text)

In [12]:
len(product_name)

80

In [13]:
#Save Data
listCols = ['product_name','product_price','product_sold']
dict_data = dict(zip(
            listCols,
                (product_name,
                 product_price,
                 product_sold)
))

In [14]:
import json 
with open('digimap.json','w') as fp:
    json.dump(dict_data, fp)

In [16]:
# if length are not equal
max_length = max(len(product_price), len(product_sold))

product_price += [None] * (max_length - len(product_price))
product_sold += [None] * (max_length - len(product_sold))

In [17]:
print(len(product_name), len(product_price), len(product_sold))

80 80 80


In [18]:
df = pd.DataFrame(data = dict_data)
df.head()

,product_name,product_price,product_sold
0,Vivo Y19 (8GB + 128GB ROM) 6.53 Inch 16MP Trip...,RM398.00,765 sold
1,Vivo Y17 6GB RAM + 128GB ROM 6.35 Inch 13MP LT...,RM353.00,848 sold
2,Vivo Y50 8GB RAM + 128GB ROM 6.53 Inches 13MP ...,RM445.00,38 sold
3,Vivo V15 8GB RAM + 256GB ROM 6.53 Inch 24MP Tr...,RM415.00,40 sold
4,Oppo A3s 4GB RAM + 64GB ROM 6.2 Inch 13MP LTE ...,RM267.00,230 sold


In [19]:
df.to_csv('digimap.csv',index=False)

##USING API

In [1]:
import requests

In [20]:
#shopee_url = 'https://shopee.co.id'
shopee_url = 'https://shopee.com.my'
keyword = 'Vivo'

header = {
    'User-Agent' : 'Chrome',
    'Referer' : '{}search?keyword={}'.format(shopee_url, keyword)
}

#url = 'https://shopee.co.id/api/v4/search/search_items?by=revelancy&keyword={}&limit=60&newest=0&order=desc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2'.format(keyword)
url = 'https://shopee.com.my/api/v4/search/search_items?by=relevancy&extra_params=%7B%22global_search_session_id%22%3A%22gs-b11c8e7a-061f-4399-8203-9dcbb23ce79e%22%2C%22search_session_id%22%3A%22ss-9553b7e8-b1d7-4ece-a190-3f37ec9350e8%22%7D&keyword={}&limit=60&match_id=285083792&newest=0&order=desc&page_type=shop&scenario=PAGE_SHOP_SEARCH&version=2&view_session_id=76f1a5e2-cf5a-4800-a566-d88d647611f9'.format(keyword)

#API request
r = requests.get(url, headers= header).json()

#Scraping
col_list, price_list, sold_list = [], [], []
for item in r['items']:
    col_list.append(item['item_basic']['name'])
    price_list.append(item['item_basic']['price'])
    sold_list.append(item['item_basic']['sold'])

KeyError: 'items'